In [1]:
import pandas as pd
import numpy as np
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings
import re

import config
from yandex_disk_func import *
from parse_functions import *



# # указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\35_mediaplan_parser'

public_key = config.public_key # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.options.mode.chained_assignment = None

In [3]:
base_cols = ['supplier', 'report_name', 'sheet_name', 'brand', 'period', 'source', 'site/ssp', 'placement', 
                     'targeting', 'geo', 'soc_dem', 'ad copy format', 'rotation type', 
                     'unit price', 'frequency', 'reach', 'impressions', 'clicks', 
             'budget_without_nds', 'budget_nds', 'views', 'vtr, %']

In [5]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # Через цикл проходим по папке с файлами
        # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # Проходим через цикл по содержимому папки (отдельный флайт)
        for i in range(len(yandex_responce['_embedded']['items'])):
            file_info = yandex_responce['_embedded']['items'][i]
            if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
                file_name = file_info['name'] # сохраняем название файла
                if 'xls' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
                    file_path = file_info['path']
                    
                    # report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
                    print(file_name)
                    res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
                    download_response = requests.get(res_file_link['href'])

/01_yandex
/02_prog
Plan_Игроник_РСХБ_beeline_Свой вклад_banners_160724 (1).xlsx
/03_Полина


In [6]:
data_link = download_response.content

In [7]:
sheet_names = pd.ExcelFile(BytesIO(data_link))

In [8]:
sheet_names.sheet_names

['Plan_Media', 'ТТ_Media', 'РЕДАКЦИОННАЯ ПОЛИТИКА БИЛАЙН']

In [26]:
sheet_name='Plan_Media'
df = pd.read_excel(BytesIO(data_link), sheet_name=sheet_name, header=None)
sheet_name = normalize_headers(sheet_name)
# забираем индекс начала таблицы
start_index = get_index_row(df, 1, 'source')
# забираем название полей из файла
col_names_list = df.iloc[start_index].fillna('').apply(normalize_headers)

ffill_columns = [1, 2, 3, 4]
df[ffill_columns] = df[ffill_columns].ffill(limit=1) # заполняем вниз
df = df.fillna('')

# сохраняем название бренда
brand = df[2].loc[get_index_row(df, 1, 'brand')] 
# сохраняем период
period = df[2].loc[get_index_row(df, 1, 'period')]
# задаем названия полей
df.columns = df.iloc[start_index].apply(normalize_headers) # забираем название полей из файла
# обрезаем верхнюю часть таблицы. она больше не нужна
df = df.iloc[start_index+2:].reset_index(drop=True)
# забираем окончание таблицы
end_index = get_index_row(df, 'source', 'итого')
# обрезаем таблицу снизу
df = df.iloc[:end_index].reset_index(drop=True)

In [27]:
df

11,,source,site/ssp,placement,targeting,ad copy format,unit quantity,rotation type,unit price,unit,"ratecard price per period (rub, net)",frequency,reach,impressions,"cpm, (rub, net)","ctr, %",cpc,clicks
0,,beeline AI,DSP,Desktop + mobile \n(+in-app),"ГЕО: РФ, кроме Крыма\nЦА: Все 25-65, ядро 30-45 (Ж 50%/М 50%)\n• Доход от 30 000 руб/мес, ядро от 70 000 руб/мес\n• Интерес к вкладам\n• Пользователи, которые интересуются банковскими продуктами, посещают разделы банковских порталов и сайтов о вкладах\n• Установлено приложение/посещают сайты/получают смс от банков-конкурентов\n• Интересуются вступлением в наследство, порядком наследования\n• Посетители, которых интересует купле-продажа недвижимости, земельных участков\n• Ключевые слова ""вклады",Banners,47348485,CPM,88,1000 imp.,4166666.68,3,15782828.333333,47348485,88.0,0.0013,67.692308,61553.0305


In [18]:
col_index

1

In [16]:
col_names_list

0                                         
1                                   source
2                                 site/ssp
3                                placement
4                                targeting
5                           ad copy format
6                            unit quantity
7                            rotation type
8                               unit price
9                                     unit
10    ratecard price per period (rub, net)
11                               frequency
12                                   reach
13                             impressions
14                         cpm, (rub, net)
15                                  ctr, %
16                                     cpc
17                                  clicks
Name: 11, dtype: object

In [ ]:
# источник Beeline
# типы размещения Видео и Баннерная реклама
# Функция для обработки медиаплана 
# 1. Медиаплан для обработки находится на листе Plan_Media 
# 2. В столбике B  находится слово Brand справа от него в столбике C название Бренда
# 3. В столбике B  находится слово Period справа от него в столбике C название указан период медиаплана
# 4. В столбике B  должно находиться поле Source
# 5. Каждая таблица должна заканчиваться строкой итогов 

def get_beeline_mediaplan(data_link, network, report_name):
    sheet_name='Plan_Media'
    df = pd.read_excel(BytesIO(data_link), sheet_name=sheet_name)
    sheet_name = normalize_headers(sheet_name)
    # заполняем вниз объединенные ячейки
    df['Unnamed: 1'] = df['Unnamed: 1'].ffill()
    df['Unnamed: 2'] = df['Unnamed: 2'].ffill()
    df['Unnamed: 3'] = df['Unnamed: 3'].ffill()
    df['Unnamed: 4'] = df['Unnamed: 4'].ffill()
    df = df.fillna('')

    # сохраняем название бренда
    brand = df['Unnamed: 2'].loc[get_index_row(df, 'Unnamed: 1', 'brand')] 
    # сохраняем период
    period = df['Unnamed: 2'].loc[get_index_row(df, 'Unnamed: 1', 'period')]
    # забираем индекс начала таблицы
    start_index = get_index_row(df, 'Unnamed: 1', 'source')

    
    # задаем названия полей
    df.columns = df.iloc[start_index].apply(normalize_headers) # забираем название полей из файла
    # обрезаем верхнюю часть таблицы. она больше не нужна
    df = df.iloc[start_index+2:].reset_index(drop=True)
    # забираем окончание таблицы
    end_index = get_index_row(df, 'source', 'итого')
    # обрезаем таблицу снизу
    df = df.iloc[:end_index].reset_index(drop=True)
    # создаем базовый список полей, которые есть всегда вне зависимости от типа размещения
    standart_columns = ['source', 'site/ssp', 'placement', 'targeting', 'ad copy format', 'unit', 'rotation type', 
          'unit price', 'frequency', 'reach', 'impressions', 'clicks', 'ratecard price per period (rub, net)']
    
    # проверяем наличие Видео размещений. Если они есть, то используем дополнительные поля из таблицы
    # если Видео размещений нет, то добавляем дополнительно 2 поля с 0 (это нужно для нормализации общей таблицы
    if 'views' not in list(df.columns):
        df['views'] = 0
        df['vtr, %'] = 0.0
        
    standart_columns += ['views', 'vtr, %']
    # оставляем только нужные поля
    df = df[standart_columns]
    df['views'] = df['views'].apply(normalize_digits)
    df['vtr, %'] = df['vtr, %'].apply(normalize_digits)
    df['views'] = df['views'].astype('int')
    df['vtr, %'] = df['vtr, %'].astype('float')
    
     # переименовываем поля
    df = df.rename(columns={'ratecard price per period (rub, net)': 'budget_without_nds'})
    # добавляем поля с общей информацией
    df['supplier'] = network
    df['report_name'] = report_name
    df['sheet_name'] = 'Plan_Media'
    df['brand'] = brand
    df['period'] = period

    # парсим Гео и Соц. дем из поля targeting
    df['geo'] = df['targeting'].apply(lambda x: get_targetings(x, 'гео:','\n', flag='geo'))
    df['soc_dem'] = df['targeting'].apply(lambda x: get_targetings(x, 'ца:','\n', flag='soc'))
    
    # убираем знак рубля, если он есть в стоимости
    currecny_columns = ['unit']
    df[currecny_columns] = df[currecny_columns].apply(get_digits)
    
    # # добавляем рассчитываемые показатели
    # # df['budget_without_nds'] = ((df['unit quantity'] / 1000) * df['unit price']).astype('float')
    df['budget_without_nds'] = df['budget_without_nds'].astype('float')
    df['budget_nds'] =(df['budget_without_nds'] * 1.2).astype('float').round(2)
    # если в этих полях встречаются пустые ячейки, то заменяем их на 0
    df['vtr, %'] = df['vtr, %'].apply(replace_blank)
    df['views'] = df['views'].apply(replace_blank)
    
    # переставляем поля местами, чтобы все было единообразно
    df = df[base_cols]
    return df

In [ ]:
network = 'weborama'
report_name = 'tst'
tmp_dict = {}
sheet_names = pd.ExcelFile(BytesIO(data_link))
# в этом файле присутствуют скрытые листы
# нам нужно исключить их из парсинга
# поэтому добавляем дополнительный блок с проверкой статуса листа
sheets = get_sheets_list(sheet_names, extention)
for sheet in sheets:
    sheet_name = check_excel_sheets(sheet, extention)
    if sheet_name:
        if 'banner' in sheet_name.lower() or 'video' in sheet_name.lower() \
        or 'баннер' in sheet_name.lower() or 'видео' in sheet_name.lower():
            
            df = pd.read_excel(BytesIO(data_link), sheet_name=sheet_name, header=None)
            sheet_name = normalize_headers(sheet_name)
            print(sheet_name)
            # заголовки в файле состоят из 2-х строк, поэтому нужно выполнить заполнение вниз на 1 строку
            # забираем название полей, в которых нужно сдвинуть строку вниз
            ffill_columns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
            df[ffill_columns] = df[ffill_columns].ffill(limit=1) # заполняем вниз
            
            # т.к. мы выполнили заполнение вниз на 1 строку
            # у нас дублируются заголовки, поэтому мы берем второе вхождение забираем индекс начала таблицы
            start_index = get_index_row(df, 1, 'период') + 1
            # забираем название полей из файла
            col_names_list = df.iloc[start_index].fillna('').apply(normalize_headers)
            pattern = 'показы'
            # получаем индекс колонки
            col_index = get_col_index(col_names_list, pattern, flag='equals')
            df[col_index] = df[col_index].fillna('0')
            df = df.fillna('')

            # сохраняем название бренда
            brand = df[1].loc[get_index_row(df, 1, 'клиент')]
            brand = brand[len('Клиент:'):].strip()
            
            # если целевая аудитория НЕ будет указана в самой таблице
            # то возьмем значение из описания НАД таблицей
            ca = df[1].loc[get_index_row(df, 1, 'ца')]
            ca = ca[len('ЦА:'):].strip()
            
                        
            # т.к. одно из полей может менять название, нам нужно его к единому стандарту
            # сначала получим индекс колонки, которая нам нужна. она тоже может менять положение (может быть втоорой или третьей)
            pattern = 'сервис'
            # получаем индекс колонки
            col_index = get_col_index(col_names_list, pattern)
            # получаем индекс строки
            # row_index = get_index_row(df, col_index, pattern) + 1
            # меняем название в нужнй ячейке
            df[col_index].loc[start_index] = 'source'
            
            # меняем назвние еще одного поля
            pattern = 'объем'
             # получаем индекс колонки
            col_index = get_col_index(col_names_list, pattern)
            # получаем индекс строки
            # row_index = get_index_row(df, col_index, pattern) + 1
            # меняем название в нужнй ячейке
            df[col_index].loc[start_index] = 'budget_without_nds'

            # меняем назвние еще одного поля
            pattern = 'частота'
             # получаем индекс колонки
            col_index = get_col_index(col_names_list, pattern)
            # получаем индекс строки
            # row_index = get_index_row(df, col_index, pattern) + 1
            # меняем название в нужнй ячейке
            df[col_index].loc[start_index] = 'frequency'
            
            # задаем названия полей
            df.columns = df.iloc[start_index].apply(normalize_headers) # забираем название полей из файла
            # обрезаем верхнюю часть таблицы. она больше не нужна
            df = df.iloc[start_index+1:].reset_index(drop=True)
            # обрезаем таблицу снизу
            end_index = get_index_row(df, 'показы', '0')
            df = df.iloc[:end_index]
            # создаем базовый список полей, которые есть всегда вне зависимости от типа размещения
            standart_columns = ['период', 'source', 'формат рекламы', 'гео', 'аудиторные данные перечислены через слэш (/)',
                            'стоимость за единицу (до ндс)', 'budget_without_nds', 'показы', 'охват', 'клики', 'frequency', 'модель закупки']
            
            # проверяем наличие Видео размещений. Если они есть, то используем дополнительные поля из таблицы
            # если Видео размещений нет, то добавляем дополнительно 2 поля с 0 (это нужно для нормализации общей таблицы     
            if 'vtr' not in list(df.columns):
                df['views'] = 0
                df['vtr'] = 0.0
            else:
                df['views'] = df['vtr'].astype('float') * df['показы'].astype('float')
                
            if 'ца' not in list(df.columns):
                df['ца'] = ca

            standart_columns += ['ца', 'views', 'vtr']
            # оставляем только нужные поля
            df = df[standart_columns]
            # приводим названия полей к единому стандарту
            df = df.rename(columns={'период': 'period', 'формат рекламы': 'ad copy format', 'гео': 'geo',
                                    'аудиторные данные перечислены через слэш (/)': 'targeting', 'ца': 'soc_dem', 
                                    'модель закупки': 'rotation type', 'стоимость за единицу (до ндс)': 'unit price',
                                    'показы': 'impressions', 'охват': 'reach', 'клики': 'clicks', 'vtr': 'vtr, %'})
            df['supplier'] = network
            df['report_name'] = report_name
            df['sheet_name'] = sheet_name
            df['brand'] = brand
            df['site/ssp'] = ''
            df['placement'] = ''
            df['budget_nds'] =(df['budget_without_nds'] * 1.2).astype('float').round(2)
            
            df = df[base_cols]

In [ ]:
df

In [ ]:
text = """Гео: РФ
Соц.дем: 25+
GPT таргетинг: МСБ: (предприниматели, бизнес, услуги банков, РКО,  инвестиции, ВЭД)
"""

In [ ]:
text

In [ ]:
text = text.lower().strip()
if 'гео:' in text:
    geo = get_target_text('гео:', '\n', text)
if 'соц:' in text:
    soc_dem = get_target_text('соц:', '\n', text)
